In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import os, warnings # 경고메세지 라이브러리
import re   # 정규표현식
import time
warnings.filterwarnings(action='ignore') # 경고메세지 무시

import pandas as pd

In [2]:
# 수집된 데이터 가져오기
dat = pd.read_excel('./lotteria_all_store.xlsx', encoding='utf-8')
# dat = dat.iloc[0:320]
# dat = dat.iloc[320:640]
dat = dat.iloc[640:960]
# dat = dat.iloc[960:]
dat

,지점명,주소,주소1,주소2,주소3
960,금강휴게소,충청북도 옥천군 동이면 금갈로 596,충청북도,옥천군,동이면 금갈로 596
961,천안두정,충청남도 천안시 서북구 봉정로 380 혜성빌딩,충청남도,천안시,서북구 봉정로 380 혜성빌딩
962,대천광장,충청남도 보령시 해수욕장2길 1(신흑동),충청남도,보령시,해수욕장2길 1(신흑동)
963,GS천안,충청남도 천안시 동남구 충절로 148번지 GS슈퍼內,충청남도,천안시,동남구 충절로 148번지 GS슈퍼內
964,대전비래,대전광역시 대덕구 비래동로 13,대전광역시,대덕구,비래동로 13
965,대전신성,대전광역시 유성구 신성남로 45(신성동),대전광역시,유성구,신성남로 45(신성동)
966,대전가장,대전광역시 서구 가장로106(롯데리아),대전광역시,서구,가장로106(롯데리아)
967,금왕,충청북도 음성군 금왕읍 무극로 250,충청북도,음성군,금왕읍 무극로 250
968,대전중촌,대전광역시 중구 목중로 22 1층 롯데리,대전광역시,중구,목중로 22 1층 롯데리
969,천안쌍용,충청남도 천안시 동남구 쌍용대로 84,충청남도,천안시,동남구 쌍용대로 84


In [3]:
addr = {'서울시':'서울특별시',
        '서울':'서울특별시',
        '부산시':'부산광역시',
        '부산':'부산광역시',
        '대구':'대구광역시',
        '인천':'인천광역시',
        '광주':'광주광역시',
        '대전시':'대전광역시',
        '대전':'대전광역시',
        '울산':'울산광역시',
        '세종':'세종특별자치시',
        '경기':'경기도',
        '강원':'강원도',
        '충북':'충청북도',
        '충남':'충청남도',
        '전북':'전라북도',
        '전남':'전라남도',
        '경북':'경상북도',
        '경남':'경상남도',
        '제주':'제주특별자치도'}

In [9]:
def change_addr(x) :
    
    s_name_ori = x['지점명']
    
    # D/T 같이 '/'이 포함된 경우 네이버에 제대로 검색되지 않음
    s_name = s_name_ori
    if '/' in s_name :
        s_name = s_name.replace('/', '')
        
    f_name = f'롯데리아 {s_name}'
    
    url = f'https://search.naver.com/search.naver?query={f_name}'
          
    driver.get(url)
    time.sleep(1)
    
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    
    # 네이버에 '롯데리아 지점명' 검색하여 주소 획득
    try :

        # case1 : 롯데리아 감일
        full_addr = soup.select_one('span.addr').text
        addr1, addr2, addr3 = full_addr.split(' ', 2)
        print(f'{s_name} : {addr1} {addr2} {addr3}')
        x[ ['주소1', '주소2', '주소3'] ] = addr1, addr2, addr3

    except Exception as e :

        print(f'{s_name} : 1. {e}')
        full_addr = ''

        try :
            store_list = soup.select('div._3ZU00')
            
            # case2 : 롯데리아 의정부화룡역
            if len(store_list) == 0 :
                full_addr = soup.select('div._1mAZf')[1].text

                full_addr = full_addr[:full_addr.rindex('지번')]
                addr1, addr2, addr3 = full_addr.split(' ', 2)
                print(f'{s_name} : {addr1} {addr2} {addr3}')
                x[ ['주소1', '주소2', '주소3'] ] = addr1, addr2, addr3

            # 검색목록이 여러개인 경우, 지점명과 일치하는 주소를 획득
            for idx, i in enumerate(store_list) :
                
                except_text = ['전북', '전남', '경북', '경남']
                
                # try 구문을 사용해도 되지만 이미 이중으로 try 선언이 되어 굳이 String으로 해결
                # 지점명에 도명이 포함된 경우, 네이버 검색 시 도명이 없이 롯데리아 진안점으로 검색됨
                # 도명을 제거하여 네이버 검색결과 중 일치하는 지점의 주소를 입력
                s_name2 = 'None 선언시 에러가 생기므로 String으로 None 선언'
                if s_name[:2] in except_text :
                    s_name2 = s_name[2:]

                # case3 : 롯데리아 전북진안, 롯데리아 강남역
                if s_name in i.a.text or s_name2 in i.a.text :
                
                    try :
                        xpath = f'//*[@id="loc-main-section-root"]/section/div/div[2]/ul/li[{idx+1}]/div[2]/div[2]/div/span/a/span[2]'
                        down = driver.find_element_by_xpath(xpath)
                    except :
                        # case4 : 롯데리아 경남함안
                        xpath = f'//*[@id="loc-main-section-root"]/section/div/div[2]/ul/li[{idx+1}]/div[1]/div[2]/div/span/a/span[2]'
                        down = driver.find_element_by_xpath(xpath)

                    down.click()
                    time.sleep(1)
                    
                    # 주소에 도로명과, 복사를 뺴고 입력
                    full_addr = driver.find_element_by_css_selector('div._2b9ic').text[3:-2]
                    addr1, addr2, addr3 = full_addr.strip().split(' ', 2)
                    print(f'{s_name} : {addr1} {addr2} {addr3}')
                    x[ ['주소1', '주소2', '주소3'] ] = addr1, addr2, addr3
                    
                    break
                    
        except Exception as e:

            print(f'{s_name} : 2. {e}')
            
    if len(full_addr) < 1 :
        # drop 처리하려 했으나 작업 편의성을 위해 None 처리
#         print(f'{s_name_ori} : 삭제')
#         dat.drop( dat.loc[ dat['지점명'] == s_name_ori ].index, axis=0, inplace=True)

        print(f'{s_name_ori} : None처리')
        pass
    else :
        return x

# 웹드라이버 실행
driver = webdriver.Chrome('../chromedriver_90')

# 주소1의 이상값을 정상값으로 변경하고 없는 지점은 None처리
dat = dat.apply(lambda x : change_addr(x), axis=1)
# dat = dat.apply(lambda x : change_addr(x) if x['주소1'] not in addr.values() else x, axis=1)

# 웹드라이버 종료
driver.close()

금강휴게소 : 충북 옥천군 동이면 금강로 596
금강휴게소 : 충북 옥천군 동이면 금강로 596
천안두정 : 1. 'NoneType' object has no attribute 'text'
천안두정 : 충청남도 천안시 서북구 봉정로 380 혜성빌딩
대천광장 : 충남 보령시 해수욕장2길 1
GS천안 : 충남 천안시 동남구 충절로 148
대전비래 : 1. 'NoneType' object has no attribute 'text'
대전비래 : 대전광역시 대덕구 비래동로 13
대전신성 : 대전 유성구 신성남로 45
대전가장 : 대전 서구 가장로 106
금왕 : 충북 음성군 금왕읍 무극로 250
대전중촌 : 1. 'NoneType' object has no attribute 'text'
대전중촌 : 대전광역시 중구 목중로 22 용신빌딩
천안쌍용 : 1. 'NoneType' object has no attribute 'text'
천안쌍용 : 충청남도 천안시 동남구 쌍용대로 84
청주하복대 : 충북 청주시 흥덕구 진재로 56
청주대학 : 충북 청주시 청원구 상당로 255
대전송촌 : 1. 'NoneType' object has no attribute 'text'
대전송촌 : 대전광역시 대덕구 동춘당로94번길 11-7 광혜빌딩
공주신관 : 충남 공주시 흑수골길 34
청주사천 : 1. 'NoneType' object has no attribute 'text'
청주사천 : 충청북도 청주시 청원구 사뜸로 49
대전송강 : 대전 유성구 구즉로 71
충주칠금 : 1. 'NoneType' object has no attribute 'text'
충주칠금 : 충청북도 충주시 칠금12길 5
진천 : 1. 'NoneType' object has no attribute 'text'
진천 : 충청북도 진천군 진천읍 중앙동로 56
대전노은 : 1. 'NoneType' object has no attribute 'text'
대전노은 : 대전광역시 유성구 은구비로 42
성환 : 충남 천안시 서북구 성

안산제일 : 경기 안산시 단원구 선부광장1로 56
안성중앙 : 경기 안성시 중앙로 397
서울랜드2호 : 경기 과천시 광명로 181
봉일천 : 경기 파주시 조리읍 봉천로 40
분당미금 : 1. 'NoneType' object has no attribute 'text'
분당미금 : 경기도 성남시 분당구 돌마로 68
시흥연성 : 1. 'NoneType' object has no attribute 'text'
시흥연성 : 경기도 시흥시 진말로19번길 16
부천상동 : 1. 'NoneType' object has no attribute 'text'
부천상동 : 경기도 부천시 상동로 87 가나베스트타운3
야탑역 : 경기 성남시 분당구 성남대로925번길 11
안산신도시 : 경기 안산시 단원구 광덕대로 154
인천청천 : 인천 부평구 마장로 396
시흥시화 : 1. 'NoneType' object has no attribute 'text'
시흥시화 : 경기도 시흥시 옥구상가2길 4
양평 : 1. 'NoneType' object has no attribute 'text'
양평 : 경기도 양평군 양평읍 역전길 3
안산초지 : 경기 안산시 단원구 광덕1로 55
남양주청학 : 경기 남양주시 별내면 청학로중앙길 15
안양역사 : 경기 안양시 만안구 만안로 232
홈플러스동수원 : 1. 'NoneType' object has no attribute 'text'
홈플러스동수원 : 경기도 수원시 팔달구 효원로 257 홈플러스동수원점
동암남부 : 인천 부평구 동암남로 5
의정부용현 : 1. 'NoneType' object has no attribute 'text'
의정부용현 : 경기도 의정부시 충의로 62
동인천역전 : 인천 중구 참외전로 122
일산탄현 : 1. 'NoneType' object has no attribute 'text'
일산탄현 : None처리
인천용현 : 1. 'NoneType' object has no attribute 'text'
인천용현 : 인천광역시 미추홀구 낙

포항 : 경상북도 포항시 북구 중앙로 282
전주고사 : 전북 전주시 완산구 전주객사5길 22-3
목포상동 : 1. 'NoneType' object has no attribute 'text'
목포상동 : 전라남도 목포시 백년대로 339
청주금천 : 1. 'NoneType' object has no attribute 'text'
청주금천 : 충청북도 청주시 상당구 수영로 285 튼튼타워
강릉교동 : 1. 'NoneType' object has no attribute 'text'
강릉교동 : 강원도 강릉시 교동광장로 106
평촌주공공이 : 경기 안양시 동안구 관평로170번길 33
부천역EXP : 경기 부천시 부천로 1
부천역 : 1. 'NoneType' object has no attribute 'text'
부천역 : 경기도 부천시 부천로 1
의정부회룡역 : 1. 'NoneType' object has no attribute 'text'
의정부회룡역 : 경기도 의정부시 평화로 374 한승아파트
인천연희DT : 인천 서구 서곶로 289
수원역광장 : 1. 'NoneType' object has no attribute 'text'
수원역광장 : 경기도 수원시 팔달구 덕영대로 924 AK플라자 수원점 1F
안산DT : 1. 'NoneType' object has no attribute 'text'
안산DT : 경기도 안산시 상록구 예술광장1로 68
안양 : 1. 'NoneType' object has no attribute 'text'
안양 : 경기도 안양시 만안구 안양로292번길 28
평택중앙 : 경기 평택시 평택2로10번길 10-3
부산사상 : 1. 'NoneType' object has no attribute 'text'
부산사상 : 부산광역시 사상구 사상로 211 롯데리아
부산역 : 1. 'NoneType' object has no attribute 'text'
부산역 : 부산광역시 동구 중앙대로 201 행운예식장
부산터미날1호 : 부산 금정구 중앙대로 2238

In [10]:
# 주소1의 줄임말을 풀어서 치환하고 None처리 된 행을 제거
dat['주소1'] = dat['주소1'].replace(addr)
dat.dropna(inplace=True)
dat.head()

,지점명,주소,주소1,주소2,주소3
960,금강휴게소,충청북도 옥천군 동이면 금갈로 596,충청북도,옥천군,동이면 금강로 596
961,천안두정,충청남도 천안시 서북구 봉정로 380 혜성빌딩,충청남도,천안시,서북구 봉정로 380 혜성빌딩
962,대천광장,충청남도 보령시 해수욕장2길 1(신흑동),충청남도,보령시,해수욕장2길 1
963,GS천안,충청남도 천안시 동남구 충절로 148번지 GS슈퍼內,충청남도,천안시,동남구 충절로 148
964,대전비래,대전광역시 대덕구 비래동로 13,대전광역시,대덕구,비래동로 13


In [ ]:
# null 확인
dat.isnull().sum()

In [ ]:
# 주소1 이상치 확인
dat['주소1'].unique()

In [ ]:
# 주소1, 주소2, 주소3 을 합쳐서 주소를 다시 입력
dat['주소'] = dat[ ['주소1', '주소2', '주소3'] ].apply(' '.join, axis=1)
dat

In [ ]:
# 전처리 된 데이터 저장
dat.to_excel('lotteria_all_store3.xlsx', index=False)
dat.to_csv('lotteria_all_store3.csv', index=False)
dat